In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.regularizers import l2, l1

Please note that the ERA5 native grid of online CDS is 0.25°x0.25° (atmosphere), 0.5°x0.5° (ocean waves), mean, spread and members: 0.5°x0.5° (atmosphere), 1°x1° (ocean waves). ERA5-Land: 0.1°x0.1°. So this will be returned by default. Our place of interest is in 40.5112, 16.3723, as a result, the grid is North:40.6, South:40.5, East: 16.3, and West: 16.2.

In [7]:
import cdsapi

c = cdsapi.Client()

c.retrieve(
    'reanalysis-era5-land',
    {
        'area': [
            40.6, 16.2, 40.5,
            16.3,
        ],
        'time': '00:00',
        'day': '01',
        'month': '01',
        'year': '2023',
        'variable': [
            '10m_u_component_of_wind', '10m_v_component_of_wind', '2m_temperature',
            'surface_pressure', 'surface_solar_radiation_downwards',
        ],
        'format': 'grib',
    },
    'download.grib')

2024-03-04 11:13:47,546 INFO Welcome to the CDS
2024-03-04 11:13:47,547 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-land
2024-03-04 11:13:47,606 INFO Request is queued


KeyboardInterrupt: 

In [14]:
import fsspec

fs = fsspec.filesystem('gs')
fs.ls('gs://gcp-public-data-arco-era5/co/')

NotImplementedError: 

In [16]:
import xarray
ml_wind = xarray.open_zarr(
    'gs://gcp-public-data-arco-era5/co/model-level-wind.zarr/',
    chunks={'time': 48},
    consolidated=True,
)
print("Model wind dataset size {} TiB".format(ml_wind.nbytes/(1024**4)))
ml_wind

ValueError: unrecognized engine zarr must be one of: ['scipy', 'store']

In [3]:
def get_weather_data(year, month, day, area, time,
                     file_location = 'weather_data.nc'
                     ):
    """
    Input:
        year, month, day, area, time, file_location: Strings
    Outputs:
        file_location: A string as in input
    """
    c = cdsapi.Client()
    c.retrieve(
        'reanalysis-era5-single-levels',{
            'product_type':'reanalysis', # This is the dataset produced by the CDS
            'variable':['2m_temperature'],
            'year': year,
            'month': month,
            'day': day,
            'area': area,
            'time': time,
            'format':'netcdf' # The format we choose to use
        },
        file_location)
    return(file_location)

In [4]:
latitude  = "52.205337"
longitude = "0.121817"

In [5]:
# Let's proceed with the request of the data:
file_location = get_weather_data(
    year  = "2019",

    # NB: Single digits days or months must have 0s in front or that will cause an error
    day   = ['01', '02', '03','04', '05', '06','07'],
    month = "04",

    # The ERA5 accept rectangular shape grid as a searching areas
    # but we can use also input a point with this system:ß
    area  = latitude +'/'+ longitude +'/'+ latitude +'/'+ longitude,

    # We can request all 24 hours of the day. The only accepted format are listed here:
    time  = ['00:00', '01:00', '02:00', '03:00', '04:00', '05:00',
             '06:00', '07:00', '08:00', '09:00', '10:00', '11:00',
             '12:00', '13:00', '14:00', '15:00', '16:00', '17:00',
             '18:00', '19:00', '20:00', '21:00', '22:00', '23:00'],
    file_location = 'first_week_Apr_2019.nc')

2024-03-03 21:57:51,888 INFO Welcome to the CDS
2024-03-03 21:57:51,889 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-03-03 21:57:52,018 INFO Request is queued


HTTPError: 404 Client Error:  for url: https://cds.climate.copernicus.eu/api/v2/tasks/63bd5087-7a75-4dfe-9210-f0b57dc5871f

In [ ]:
import netCDF4 as nc
ds = nc.Dataset('/Users/roozbeh/Documents/GitHub/SEST/download_Location_1/Wind_WFDE5_CRU_200901_v2.1.nc')

In [6]:
import cdsapi

c = cdsapi.Client()

c.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'format': 'netcdf',
        'variable': [
            '2m_dewpoint_temperature',
        ],
        'year': [
            '2020',
        ],
        'month': [
            '01', 
        ],
        'day': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
            '13', '14', '15',
            '16', '17', '18',
            '19', '20', '21',
            '22', '23', '24',
            '25', '26', '27',
            '28', '29', '30',
            '31',
        ],
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'area': [
            40.5, 16.25, 16.5,
            40.25,
        ],
    },
    'download.nc')

2024-03-04 10:25:16,117 INFO Welcome to the CDS
2024-03-04 10:25:16,117 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-03-04 10:25:16,300 INFO Request is queued


HTTPError: 404 Client Error:  for url: https://cds.climate.copernicus.eu/api/v2/tasks/b5a92d6b-c9f8-45df-8159-d440c46db3c3

In [ ]:
import cdstoolbox as ct

# Initialise an application 
@ct.application(title='Extract a time series and plot graph')

# Add output widget to the application
@ct.output.livefigure()

# Create a function that retrieves the Near Surface Air Temperature dataset, extracts data from a point and plots the line on the livefigure
def plot_time_series():

    # Retrieve a variable over a defined time range
    data = ct.catalogue.retrieve(
        'reanalysis-era5-single-levels',
        {
            'variable': '2m_temperature',
            'product_type': 'reanalysis',
            'year': [
                '2008'
            ],
            'month': [
                '01', '02', '03', '04', '05', '06',
                '07', '08', '09', '10', '11', '12'
            ],
            'day': [
                '01', '02', '03', '04', '05', '06',
                '07', '08', '09', '10', '11', '12',
                '13', '14', '15', '16', '17', '18',
                '19', '20', '21', '22', '23', '24',
                '25', '26', '27', '28', '29', '30',
                '31'
            ],
            'time': ['00:00', '06:00', '12:00', '18:00'],
            'grid': ['3', '3']
        }
    )

    # Select a location, defined by longitude and latitude coordinates
    data_point = ct.geo.extract_point(
        data,
        lon=75.0,
        lat=43.0
    )

    # Compute the daily mean for the selected data
    data_daily = ct.climate.daily_mean(data_point)

    # Show the result as a time-series on an interactive chart
    figure = ct.chart.line(data_daily)

    return figure


In [ ]:
df = pd.read_csv('/Users/roozbeh/Library/CloudStorage/GoogleDrive-rouzbeh.shirvani01@universitadipavia.it/My Drive/Pavia/EDPR/Data/Elena/PVGIS_NEW.csv')

# Pre-processing

In [ ]:
def preprocess_data(df, start, end, prediction_feature, train_size, validation_size, scaler):
    # Convert 'time' to datetime and set as index
    df['time'] = df['time'].str.replace(' UTC', '', regex=False)
    df['time'] = pd.to_datetime(df['time'], utc=True)
    df = df.set_index('time')

    # Filter data based on start and end date
    df_filtered = df.loc[start:end]

    # Calculate daily averages
    daily_averages = df_filtered.resample('D').mean()

    # Initialize lists to hold the features and labels
    train_features = []
    train_labels = []

    # Loop over each day to align daily averages with hourly values
    for i in range(len(daily_averages)):
        daily_avg = daily_averages.iloc[i]
        hourly_values = df_filtered[prediction_feature][i*24:(i+1)*24]

        if len(hourly_values) == 24:
            train_features.append(daily_avg.values)
            train_labels.append(hourly_values.values)

    # Calculate indices for splitting
    total_samples = len(train_features)
    train_end = int(total_samples * train_size)
    val_end = train_end + int(total_samples * validation_size)

    # Split the dataset
    train_feat, train_lab = train_features[:train_end], train_labels[:train_end]
    val_features, val_labels = train_features[train_end:val_end], train_labels[train_end:val_end]
    test_features, test_labels = train_features[val_end:], train_labels[val_end:]

    # Stack the features and labels
    train_feat = np.vstack(train_feat)
    train_lab = np.vstack(train_lab)
    val_features = np.vstack(val_features)
    val_labels = np.vstack(val_labels)
    test_features = np.vstack(test_features)
    test_labels = np.vstack(test_labels)

    # Fit the scaler on the training features and labels
    scaler.fit(train_feat)
    label_scaler = MinMaxScaler()
    label_scaler.fit(train_lab)

    # Scale the features and labels
    train_features_scaled = scaler.transform(train_feat)
    train_lab_scaled = label_scaler.transform(train_lab)
    test_features_scaled = scaler.transform(test_features)
    test_labels_scaled = label_scaler.transform(test_labels)
    val_features_scaled = scaler.transform(val_features)
    val_labels_scaled = label_scaler.transform(val_labels)

    return train_features_scaled, train_lab_scaled, test_features_scaled, test_labels_scaled, val_features_scaled, val_labels_scaled

In [ ]:
def train_nn_model(train_features_scaled, train_lab_scaled, neu_number, activation, optimizer='adam', loss='mae', epochs=300, batch_size=10, l2_factor=None, l1_factor=None):
    # Define the kernel regularizer based on l2_factor and l1_factor
    if l2_factor and l1_factor:
        print('Select only one regulaizer')
        pass
    elif l2_factor:
        kernel_regularizer = l2(l2_factor)
    elif l1_factor:
        kernel_regularizer = l1(l1_factor)
    else:
        kernel_regularizer = None

    # Define the neural network model
    model = Sequential([
        Dense(units=neu_number, activation=activation, input_shape=(train_features_scaled.shape[1],), kernel_regularizer=kernel_regularizer),
        Dense(units=24)  # Assuming the output layer has fixed 24 units
    ])

    # Compile the model
    model.compile(optimizer=optimizer, loss=loss)

    # Train the model
    history = model.fit(train_features_scaled, train_lab_scaled, epochs=epochs, batch_size=batch_size, verbose=1)

    return history

# Solar Irradiation

In [ ]:
start = '2005-01-01 00:00:00'
end = '2015-12-31 23:00:00'
prediction_feature = 'Gd(i)'
train_size = 0.7
validation_size = 0.15
scaler = MinMaxScaler()

train_features, train_labels, test_features, test_labels, val_features, val_labels = preprocess_data(
    df, start, end, prediction_feature, train_size, validation_size, scaler)

In [ ]:
# Example usage
neu_number = 20
activation = 'relu'
optimizer = 'adam'
loss = 'mae'
epochs = 300
batch_size = 10
l1_factor = 0.05  # Optional

# Assuming train_features_scaled and train_lab_scaled are already defined and available
history = train_nn_model(train_features, train_labels, neu_number, activation, optimizer, loss, epochs, batch_size, l1_factor)